In [ ]:
!pip3 install requests_futures
import logging
import logging.config
import os
import re

import requests
import requests_futures
from requests_futures.sessions import FuturesSession
import numpy as np
import pandas as pd
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

logger = logging.getLogger(__name__)

### Update variables
1. Update `site_id`,prefix, if data is in a different organization and site.   
2. HPU_Envs.json to store the environment varaibles, upload HPU_envs.json into ws project data assets. The format of the HPU_Envs.json is the following:
```
{
   "Instruction":{
      "MX_BASE_URL":"Extract health or manage host, and replace ******* in https://*******/maximo/, e.g Get main.manage.ivt13rel88.ivt.suite.maximo.com from https://main.manage.ivt13rel88.ivt.suite.maximo.com/maximo/oslc/graphite/relengineer/index.html",
      "MX_APIKEY":"Application Administration -> Go To Administration -> Integration -> API Keys -> Copy key from admin user card (e.g. 6805t46gn3tef37pu0picpg9vcq3hsmamm1enc43), or Add API key if not exist."
   },
   "MX_APIKEY":"**************",
   "MX_BASE_URL":"https://*******/maximo/"
}
```

3. Update `model_version` based on HPU version. In MAS8.7, it's 4.0.0, in MAS8.8, it's 5.0.0, in MAS8.9, it's 6.0.0   
4. Optional:Update ASSET_CLASS if needed based on the data loaded in 3.a.    
E.g ASSET_CLASS has thress samples, when only load data for Substation Transformer, use first ASSET_CLASS sample, when using the demo data set, use second ASSET_CLASS sample. 

         

In [ ]:
global MX_BASE_URL,MX_API_CONTEXT,MX_USER,MX_PWD,MX_APIKEY,APIKEY

import json
# Opening JSON file
f = open('/project_data/data_asset/HPU_Envs.json',)
data = json.load(f)
f.close()
os.environ['MX_BASE_URL'] = data['MX_BASE_URL']
os.environ['MX_APIKEY'] = data['MX_APIKEY']

site_id="EUDEMO"
prefix="FUTURE"
#### 6.0.0 for MAS8.9
model_version="7.0.0"   


####------For ST future Demo------
ASSET_CLASS = [
  {
    "AssetType": "SUBSTATIONTRANSFORMER",
    "ShortName": "ST",
    "ModelName": "IBM future score from csv sample "+model_version+" demo"
  }]

print(ASSET_CLASS)

In [ ]:
def _call_maximo(url_path, method, json=None, headers=None, **kwargs):
    global MX_USER 
    global MX_PWD
    global MX_APIKEY
    
    MX_APIKEY = os.environ['MX_APIKEY']

    url=""
    baseurl = os.environ['MX_BASE_URL']
   
    if headers is None:
        headers = {}
    
    if MX_APIKEY is None:
        api_context = 'oslc'
#         print(baseurl)
        url = '%s%s%s&%s&%s&%s' % (baseurl, api_context, url_path,'_lid='+MX_USER,'_lpwd='+MX_PWD,'lean=1')
    else:
        api_context = 'api'
        url = '%s%s%s' % (baseurl, api_context, url_path)
        if MX_APIKEY is not None:
            headers['apikey'] = MX_APIKEY

    ssl_verify = False
    kwargs['ssl_verify'] = ssl_verify
    
    print(url)
#     print(headers)
    print(json)
    return _call(url=url, method=method, json=json, headers=headers, timeout=30000, **kwargs)

def _call(url, method, json=None, headers=None, **kwargs):
    return api_request(url, method=method, headers=headers, json=json, **kwargs)

def api_request(url, method='get', headers=None, json=None, timeout=None, ssl_verify=True, session=None, **kwargs):
    ssl_verify = False
    timeout=300
    logger.info('method=%s, url=%s, headers=%s, timeout=%s, ssl_verify=%s, json=%s, session=%s, kwargs=%s' % (method, url, str(headers), timeout, ssl_verify, str(json), str(session), str(kwargs)))

    r = requests if session is None else session

    if method == 'get':
        resp = r.get(url, headers=headers, verify=ssl_verify, json=json, timeout=timeout, **kwargs)
    elif method == 'post':
        resp = r.post(url, headers=headers, verify=ssl_verify, json=json, timeout=timeout, **kwargs)
    elif method == 'put':
        resp = r.put(url, headers=headers, verify=ssl_verify, json=json, timeout=timeout, **kwargs)
    elif method == 'patch':
        resp = r.patch(url, headers=headers, verify=ssl_verify, json=json, timeout=timeout, **kwargs)
    elif method == 'delete':
        resp = r.delete(url, headers=headers, verify=ssl_verify, json=json, timeout=timeout, **kwargs)
    else:
        raise ValueError('unsupported_method=%s' % method)

    if isinstance(r, FuturesSession):
        return resp

    logger.info('resp.status_code=%s, method=%s, url=%s' % (str(resp.status_code), method, url))

    if resp.status_code not in (requests.codes.ok, requests.codes.created, requests.codes.accepted, requests.codes.no_content):
        logger.warning('error api request: url=%s, method=%s, status_code=%s, response_text=%s' % (url, method, str(resp.status_code), str(resp.text)))
        return None

    return resp


In [ ]:
#Create query
def create_query(clausename,targetObject,oslcWhereStatement):
    objectStructure = "MXAPIASSET" if targetObject=="ASSET" else "MXAPIOPERLOC"    
    querypayload={
        "ispublic":True,
        "clausename":clausename
    }
    resp = _call_maximo(url_path='/os/'+objectStructure+'?lean=1&oslc.where='+oslcWhereStatement+'&action=savethisquery', method='post',json=querypayload)
    print(resp)

In [ ]:
#Get scores list and active scores
def getScoresForTheGroup(group_restid):   
    resp = _call_maximo(url_path='/os/MXAPIEXPGROUP/'+group_restid+'/ahmethodology?oslc.select=scoretype%2Cahmethodologyid%2Capmscoreobjectattr.title%2Cdescription%2Cactive%2Cscoredependencies&oslc.pageSize=20&oslc.orderBy=%2Bahmethodologyid&searchAttributes=description&collectioncount=1&ignorecollectionref=1&relativeuri=1&addschema=1&lean=1&internalvalues=1', method='get')
#     print(resp.json())
    return resp.json()

def getScoreRestId(ahmethodologyid):   
    resp = _call_maximo(url_path='/os/mxapiahmethodology?lean=1&oslc.select=href,ahmethodologyid&oslc.where=ahmethodologyid='+str(ahmethodologyid)+'', method='get')
    data = resp.json()
#     print(data['member'][0]['href'].split("/")[-1:][0])
    return data['member'][0]['href'].split("/")[-1:][0]

def activeScore(score_restid,active):
    payload={
        "active":active
    }
    headers = {"patchtype": "MERGE"}
    headers ["x-method-override"] = "PATCH"

    resp = _call_maximo(url_path='/os/mxapiahmethodology/'+score_restid+'?lean=1', method='post',headers=headers,json=payload)
    print(resp)

In [ ]:
#Create asset group
def create_hpu_asset_group(assetGroupLabel,clausename,targetObject,model):
    objectStructure = "MXAPIASSET" if targetObject=="ASSET" else "MXAPIOPERLOC"    
    payload={
        "usewith":targetObject,
        "intobjectname":objectStructure,
        "egtype":"QUERY",
        "clausename":clausename,
        "usedby":"SCORE",
        "grouplabel":assetGroupLabel,
        "groupcalctype":"NOTEBOOK",
        "extmodelname":model
    }
    headers = {"properties": "expgroupid,grouplabel,expgroupname,extmodelname,description,clausename,egtype,usewith,usedby,groupcalctype"}
    
    resp = _call_maximo(url_path='/os/MXAPIEXPGROUP?lean=1', method='post',headers=headers,json=payload)
    print(resp.json())
    return resp.json()
    
def createScoresFromTemplateForGroup(expgroupname):
    payload={
        "expgroupname":expgroupname
    }
    headers = {"patchtype": "MERGE"}
    resp = _call_maximo(url_path='/service/ahinsight?action=wsmethod%3AcreateScoresFromTemplateForGroup&interactive=1&lean=1&relativeuri=1&internalvalues=1', method='post',json=payload)
    print(resp)


#####For clean up#####
def get_asset_group_info(grouplabel):
    if grouplabel is None:
        raise ValueError('the given argument grouplabel is None')
    resp = _call_maximo(url_path='/os/MXAPIEXPGROUP?lean=1&addid=1&oslc.where=grouplabel=%22'+grouplabel+'%22&oslc.select=*', method='get')
    if resp is not None:
        data = resp.json()
#         print(data)
        if len(data['member']) != 0:
            return (data['member'][0])
    return None

def delete_asset_group(grouplabel):
    if grouplabel is None:
        raise ValueError('the given argument grouplabel is None')
    group = get_asset_group_info(grouplabel)
    if group is not None:
        scores = getScoresForTheGroup(group['href'].split("/")[-1:][0])['member']
        scores.reverse()
        print(scores)
        for s in scores:
            ahmethodologyid = s['ahmethodologyid']
            #get score restid
            scoreRestid=getScoreRestId(ahmethodologyid)

            #active score descend
            activeScore(scoreRestid,False)            
        href = group['href']
        url_path=href.split("/api")[1]
        print(url_path)
        resp = _call_maximo(url_path=url_path, method='delete')
        print(resp)
    

In [ ]:
for obj in ['ASSET']:
    for item in ASSET_CLASS:  
        print('\nCreate query for:')
        print(item)

        if 'ShortName' in item:
    #         classstructure.classificationid="METALSUPPORTSTRUCTURE" and siteid="TDIST"
            clausename = prefix+"_"+site_id+"_"+item["ShortName"]
            whereStatement = 'classstructure.classificationid=%22'+item['AssetType']+'%22%20and%20siteid=%22'+site_id+'%22'

            create_query(clausename,obj,whereStatement)
        else:
            if 'Subtypes' in item:
                for subtype in item['Subtypes']:
    #                 classstructure.classificationid="UNDERGROUNDTRANSMISSIONCABLE" and assetspec.assetattrid="SUBTYPE" and assetspec.alnvalue="Self Contained Fluid Filled (SCFF) Cables" and siteid="TDIST"
                    clausename = prefix+"_"+site_id+"_"+subtype["ShortName"]
                    whereStatement = ""
                    if obj=="ASSET":
                        whereStatement = 'classstructure.classificationid=%22'+item['AssetType']+'%22%20and%20assetspec.assetattrid=%22SUBTYPE%22%20and%20assetspec.alnvalue=%22'+subtype['Subtype']+'%22%20and%20siteid=%22'+site_id+'%22'
                    else:
                        whereStatement = 'classstructure.classificationid=%22'+item['AssetType']+'%22%20and%20locationsspec.assetattrid=%22SUBTYPE%22%20and%20locationsspec.alnvalue=%22'+subtype['Subtype']+'%22%20and%20siteid=%22'+site_id+'%22'

                    create_query(clausename,obj,whereStatement)
                
                
        

In [ ]:
#Delete group
for obj in ['ASSET']:
    for item in ASSET_CLASS:  
        print(item)
        if 'ShortName' in item and 'ModelName' in item:              
            group = prefix+"_"+site_id+"_"+item["ShortName"]+"_"+obj[0]
            print('\nDelete score group for:'+group)

            delete_asset_group(group)
        else:
            if 'Subtypes' in item:
                for subtype in item['Subtypes']:
                    if 'ModelName' in subtype:                 
                        group = prefix+"_"+site_id+"_"+subtype["ShortName"]+"_"+obj[0]
                        print('\nDelete score group for:'+group)

                        delete_asset_group(group)

                        
for obj in ['ASSET']:
    for item in ASSET_CLASS:  
        print('\nCreate score group for:')
        print(item)
        if 'ShortName' in item and 'ModelName' in item:    
            clausename = prefix+"_"+site_id+"_"+item["ShortName"]
            group = clausename+"_"+obj[0]

            #Create group
            scObj = create_hpu_asset_group(group,clausename,obj,item['ModelName'])

            #Save notebook detail for this group
            expgroupname = scObj['expgroupname']
            createScoresFromTemplateForGroup(expgroupname)

            #Get and active scores
            href = scObj['href'].split("/");
            print(href[-1:])
            group_restid = href[-1:][0]

            scores = getScoresForTheGroup(group_restid)['member']
            for s in scores:
                ahmethodologyid = s['ahmethodologyid']
                print(s['ahmethodologyid'])
                #get score restid
                scoreRestid=getScoreRestId(ahmethodologyid)

                #active score
                print("Start to active score...")
                activeScore(scoreRestid,True) 
            #We can't start analysis for all assets all together.
            expgroupnamePayload={
                "expGroupName": expgroupname
                }     
            resp =_call_maximo(url_path='/service/ahinsight?action=wsmethod%3ArecalculateScoringGroups', method='post',json=expgroupnamePayload)
            print(resp)           
        else:
            if 'Subtypes' in item:
                for subtype in item['Subtypes']:
                    if 'ModelName' in subtype:
                        clausename = prefix+"_"+site_id+"_"+subtype["ShortName"]
                        group = clausename+"_"+obj[0]

                        #Create group
                        scObj = create_hpu_asset_group(group,clausename,obj,subtype['ModelName'])

                        #Save notebook detail for this group
                        expgroupname = scObj['expgroupname']
                        createScoresFromTemplateForGroup(expgroupname)

                        #Get and active scores
                        href = scObj['href'].split("/");
                        print(href[-1:])
                        group_restid = href[-1:][0]

                        scores = getScoresForTheGroup(group_restid)['member']
                        for s in scores:
                            ahmethodologyid = s['ahmethodologyid']
                            print(s['ahmethodologyid'])
                            #get score restid
                            scoreRestid=getScoreRestId(ahmethodologyid)

                            #active score
                            print("Start to active score...")
                            activeScore(scoreRestid,True) 

                        #We can't start analysis for all assets all together.
                        expgroupnamePayload={
                            "expGroupName": expgroupname
                            }     
                        resp =_call_maximo(url_path='/service/ahinsight?action=wsmethod%3ArecalculateScoringGroups', method='post',json=expgroupnamePayload)
                        print(resp)
